<a href="https://colab.research.google.com/github/kridtapon/Vortex-TRIX-Momentum-VTM-/blob/main/Vortex_TRIX_Momentum_(VTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Vortex Indicator (VI)
def calculate_vi(df, period=14):
    """
    Calculate Vortex Indicator (VI+ and VI-).
    """
    tr = df['High'].combine(df['Close'].shift(), max) - df['Low'].combine(df['Close'].shift(), min)
    vm_plus = abs(df['High'] - df['Low'].shift())
    vm_minus = abs(df['Low'] - df['High'].shift())
    vi_plus = vm_plus.rolling(window=period).sum() / tr.rolling(window=period).sum()
    vi_minus = vm_minus.rolling(window=period).sum() / tr.rolling(window=period).sum()
    return vi_plus, vi_minus

# Function to calculate TRIX (Triple Exponential Moving Average)
def calculate_trix(df, length=15):
    """
    Calculate TRIX indicator.
    """
    ema1 = df['Close'].ewm(span=length, adjust=False).mean()
    ema2 = ema1.ewm(span=length, adjust=False).mean()
    ema3 = ema2.ewm(span=length, adjust=False).mean()
    trix = (ema3 - ema3.shift(1)) / ema3.shift(1)
    return trix

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Vortex Indicator (VI+ and VI-)
df['VI+'], df['VI-'] = calculate_vi(df, period=14)

# Calculate TRIX
df['TRIX'] = calculate_trix(df, length=15)

# Define Entry and Exit signals based on VI and TRIX
df['Entry'] = (df['VI+'] > df['VI-']) & (df['TRIX'] > df['TRIX'].shift(1))
df['Exit'] = (df['VI-'] > df['VI+']) & (df['TRIX'] < df['TRIX'].shift(1))

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           184684.367321
Total Return [%]                        84.684367
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                       8948.571658
Max Drawdown [%]                        56.404897
Max Drawdown Duration                       863.0
Total Trades                                   37
Total Closed Trades                            37
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            37.837838
Best Trade [%]                          52.322204
Worst Trade [%]                         -9.717344
Avg Winning Trade [%]                   14.705325
Avg Losing Trade [%]                    -4.884765
